external_array.npy must have the same shape as the input to the model, and have the same number of parameters as the model was trained on.  
For example, the third dimension (inner most array) carries each paramter for a specific time:  
foo = [[['p10', 'p20'], ['p11', 'p21'],['p12', 'p22'] ,['p13', 'p23']]]  
  
The sample rate and length in seconds are up to you, and must be provided in the arguments to generate.py

In [1]:

condsr = 1000 # the sample rate for the generated array to pass to generate.py

numsecs = 1 # number of seconds to generate

In [2]:
numDataPoints=numsecs*condsr

In [3]:
# save numpy array as npy file
import numpy as np
from numpy import asarray
from numpy import save
# define data
# data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

from matplotlib import pyplot as plt


In [4]:
#utility
def map(x,a,b,m,n) :
    return m + (n-m)*(x-a)/(b-a)

In [5]:
# for plotting in time
t_values = np.linspace(0, numsecs, numDataPoints) #linspace is inclusive of endpoints


In [6]:
# functions for conditioning parameter gestures
# These control sequences get upsampled to audiosr in the generate.py code, so don't need high sr here.
def linear(x, m, b=0) :
    return m*x+b  

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def step(x, a, b, m, n, numsteps) :  # 1 means function in constant, 
    assert numsteps > 0
    stepLen=(b-a)/numsteps
    stepNum=(x-a)//stepLen
    stepHeight=467 if (numsteps == 0) else (n-m)/(numsteps-1)
    return m+stepNum*stepHeight


'''
arr elements must be in [plow, phigh] 
'''
def normedNoteSeq(arr, plow, phigh, samps) :
    farr=[(n-plow)/(phigh-plow) for n in arr]
    alen=len(farr)
    fsampspernote=samps/alen
    # each note will have an duration equal to the rest within 1 sample
    return [farr[int((n)//fsampspernote)] for n in range(samps)]

'''
This creates a sequence of length sum(durseq), where each element in shorseq 
is duplicated by the correspeonding number in durseq.
'''
def spread(shortseq,durseq) :
    out=np.empty(0)
    for n in range(len(durseq)) :
        out = np.append(out, np.full(durseq[n],shortseq[n]))
    return out


'''
This creates a sequence of linear 0-1 interpolates of durations specified by durseq.
'''
def envSeq(durseq) :
    out=np.empty(0)
    for n in range(len(durseq)) :
        out = np.append(out, np.linspace(0,1,durseq[n],True))
    return out
        

In [7]:
# This is for the "Inception" experiment with RNNs and GANs
#NOTE : only generating one data point for each parameter setting
#noteseq=[64,65,66,67,68,69,70,71,72,73,74,75,76]
###noteseq=[64,71]
###numInstSteps=900 # should be divisible by numInstClasses to get equal numbr of elements in each
###numInstClasses=3
noteseq=[60,61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,72]
numInstSteps=11 # should be divisible by numInstClasses to get equal numbr of elements in each
numInstClasses=11
for n in noteseq :
    normednote=map(n,56,76,0,1) #for param file
    for inst in np.linspace(0,1,numInstSteps,True) :
        data=[[[ normednote,inst], [ normednote,inst]]] #2 data points, both the same for beg and end
        # revers inst/pitch order to match GAN output file name convention
        outdir = f'scratch/timbreInterp/inst_{int(20*inst)}_pitch_{n-56}.npy'
        print(f'will write to dir {outdir}')
        save(outdir, data)


will write to dir scratch/timbreInterp/inst_0_pitch_4.npy
will write to dir scratch/timbreInterp/inst_2_pitch_4.npy
will write to dir scratch/timbreInterp/inst_4_pitch_4.npy
will write to dir scratch/timbreInterp/inst_6_pitch_4.npy
will write to dir scratch/timbreInterp/inst_8_pitch_4.npy
will write to dir scratch/timbreInterp/inst_10_pitch_4.npy
will write to dir scratch/timbreInterp/inst_12_pitch_4.npy
will write to dir scratch/timbreInterp/inst_14_pitch_4.npy
will write to dir scratch/timbreInterp/inst_16_pitch_4.npy
will write to dir scratch/timbreInterp/inst_18_pitch_4.npy
will write to dir scratch/timbreInterp/inst_20_pitch_4.npy
will write to dir scratch/timbreInterp/inst_0_pitch_5.npy
will write to dir scratch/timbreInterp/inst_2_pitch_5.npy
will write to dir scratch/timbreInterp/inst_4_pitch_5.npy
will write to dir scratch/timbreInterp/inst_6_pitch_5.npy
will write to dir scratch/timbreInterp/inst_8_pitch_5.npy
will write to dir scratch/timbreInterp/inst_10_pitch_5.npy
will wr

In [8]:
#test
# load numpy array from npy file
from numpy import load
# load array
datain = load('scratch/timbreInterp/inst_4_pitch_16.npy')
#datain=load('scratch/GANsynthArpeggio_basic_1000samps.npy')
numParams=len(datain[0,0])
numParams

2

In [9]:
datain

array([[[0.8, 0.2],
        [0.8, 0.2]]])

In [10]:
datain[0,0]

array([0.8, 0.2])